# Social Distancing: Do we listen to eachother
Using data from [New York City's Open Data Portal](https://opendata.cityofnewyork.us/), this is an investigation of if people when encountered by city ambassadors comply with social distancing policies. 

## Part One: Data Collection & Cleaning

There are 5 datasets regarding this issue:

  1. Park Patrol: data collected from Urban Park Rangers and the Parks Enforcement Patrol (`patrol`)
  2. Parks Ambassador: data collected from Social Distancing Ambassadors (`sda`)
  3. City Wide Amabassador: data collected from Interagency Ambassadors (`citywide`)
  4. Parks Crowds Data: data collected from NYC Parks Maintenance & Operations staff (`mando`)
  5. Parks Areas: Geographical/geometric and zone data about the parks (`area`)

**Step 1: Load data from the CSV files from Open Data Portal:**

In [1]:
import pandas as pd
import numpy as np

In [2]:
patrol = pd.read_csv('./data/Parks_Patrol.csv', 
                     dtype={'simplified_encounter_type': str, 'closed_patroncount': str,
                           'closed_education': str, 'closed_outcome_spec': str, 'closed_outcome': str,
                           'closed_pdcontact': str,'sd_amenity': str, 'closed_amenity': str, 'sd_pdcontact': str})

patrol.drop(['summonscount_a01','summonscount_a03','summonscount_a04','summonscount_a22',
             'other_summonscount', 'closed_amenity', 'closed_patroncount', 'closed_education',
             'closed_outcome', 'closed_pdcontact', 'closed_outcome_spec',
             'patrol_method', 'visit_reason'], axis=1, inplace=True)

patrol.drop(patrol.loc[patrol.simplified_encounter_type == 'No Encounter'].index,inplace=True)
patrol.dropna(axis=0,subset=['sd_patronscomplied'],inplace= True)

patrol.rename(columns={'park_division':'city_agency'})
patrol['source'] = pd.Series(['patrol' for x in range(0,patrol.shape[0])])

In [3]:
sda = pd.read_csv('Parks_Ambassador.csv',
                 dtype={'simplified_encounter_type': str, 'closed_patroncount': str,
                           'closed_education': str, 'closed_outcome_spec': str, 'closed_outcome': str,
                           'closed_pdcontact': str,'sd_amenity': str, 'closed_amenity': str, 'sd_pdcontact': str})

sda.drop(['closed_amenity', 'closed_patroncount','closed_outcome', 'closed_approach', ], axis=1, inplace=True)

sda.drop(sda.loc[sda.simplified_encounter_type == 'No Encounter'].index,inplace=True)
sda.dropna(axis=0,subset=['sd_patronscomplied'],inplace= True)

sda.rename(columns={'park_division':'city_agency'})
sda['source'] = pd.Series(['sda' for x in range(0,sda.shape[0])])

In [4]:
citywide = pd.read_csv('Citywide.csv',
                 dtype={'simplified_encounter_type': str, 'closed_patroncount': str,
                           'closed_education': str, 'closed_outcome_spec': str, 'closed_outcome': str,
                           'closed_pdcontact': str,'sd_amenity': str, 'closed_amenity': str, 'sd_pdcontact': str})

citywide.drop(['closed_amenity', 'closed_patroncount','closed_outcome', 'closed_approach', ], axis=1, inplace=True)

#citywide.drop(sda.loc[citywide.simplified_encounter_type == 'No Encounter'].index,inplace=True)
citywide.dropna(axis=0,subset=['sd_patronscomplied'],inplace= True)

citywide['source'] = pd.Series(['citywide' for x in range(0,citywide.shape[0])])

In [30]:
mando = pd.read_csv('Maintenance_and_operations.csv')

mando.drop(['park_district'], axis=1, inplace=True)

mando['source'] = pd.Series(['mando' for x in range(0,mando.shape[0])])
mando.drop(mando.loc[mando.action_taken == 'Did not approach the crowd; the crowd remains'].index,inplace=True)
#mando.dropna(axis=0,subset=['sd_patronscomplied'],inplace= True)

mando.rename(columns={'amenity':'sd_amenity','action_taken':'encounter_type',
                      'encounter_timestamp':'encounter_datetime'},inplace= True)

mando['sd_patronscomplied'] = 0
mando['sd_patronsnocomply'] = 0

mando.sd_patronsnocomply = np.where(mando.encounter_type == 'Approached the crowd; they ignored the employee',
                                 mando.patroncount,0)
mando.sd_patronscomplied = np.where(mando.encounter_type == 'Approached the crowd; they complied with instructions',
                                 mando.patroncount,0)

In [33]:
#merging
patrol.encounter_datetime

39       07/08/2020 08:15:00 AM
52       07/07/2020 11:00:00 PM
53       07/07/2020 11:00:00 PM
54       07/07/2020 11:00:00 PM
60       07/07/2020 07:37:00 PM
                  ...          
47726    09/13/2020 06:00:00 PM
47754    09/13/2020 04:49:00 PM
47848                       NaN
47894    09/13/2020 04:45:00 PM
47910    09/08/2020 05:30:00 PM
Name: encounter_datetime, Length: 3476, dtype: object